In [1]:
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import MaaSSim

In [2]:
#!pip install MaaSSim
import MaaSSim.utils
import logging
from dotmap import DotMap
import MaaSSim.simulators
from MaaSSim.data_structures import structures as inData
from MaaSSim.acceptance import f_decline, f_decline_R50
from MaaSSim.acceptance import RA_kpi_veh, RA_kpi_pax

In [3]:
import osmnx as ox
import networkx as nx
import random
import pandas as pd
import matplotlib.pyplot as plt          
import numpy as np
import seaborn as sns

In [4]:
%load_ext autoreload
%autoreload 2

In [22]:
# Do not forget to set the radius for centeral nodes calculation
# The minimum trip distance is considered 1000m and this can be modified through utils.py: generate_demand function. 
# long attribute should be calculated according to the size of city
# Platform fare is 2$ per km

params = MaaSSim.utils.get_config('glance.json')
params.vehicle_fixed_positions = True
params.times.patience = 600 #in second
params.dist_threshold = 10000 #in meter
params.dist_threshold_min = 1000
params.city = "Delft, Netherlands"
params.paths.G = "../../data/graphs/Delft.graphml"
params.paths.skim = "../../data/graphs/Delft.csv"

params.d2d.fuel_cost = 0.25 #euro/km
params.platforms.comm_rate = 0.25
params.platforms.base_fare = 2
params.platforms.fare = 1.2 
params.platforms.min_fare = 2
params.speeds.ride=10
params.times.decline=20
params.assert_me = False

params.nP = 20
params.nV = 2
params.simTime = 1

In [23]:
sim = MaaSSim.simulators.simulate(params=params, f_driver_decline=f_decline, kpi_veh=RA_kpi_veh, kpi_pax=RA_kpi_pax)

09-06-22 21:13:27-WARNING-Setting up 1h simulation at 2022-06-09 20:45:57 for 2 vehicles and 20 passengers in Delft, Netherlands
09-06-22 21:13:28-INFO--------------------	Starting simulation	-------------------
09-06-22 21:13:28-INFO-pax    0  got to dest at                           2022-06-09 20:54:49
09-06-22 21:13:28-INFO-pax    2  has no offers                            2022-06-09 21:03:09
09-06-22 21:13:28-INFO-pax    2  lost his patience and left the system    2022-06-09 21:03:09
09-06-22 21:13:28-INFO-pax    3  got to dest at                           2022-06-09 21:03:30
09-06-22 21:13:28-INFO-pax    1  got to dest at                           2022-06-09 21:09:07
09-06-22 21:13:28-INFO-pax    4  got to dest at                           2022-06-09 21:15:30
09-06-22 21:13:28-INFO-pax    7  got to dest at                           2022-06-09 21:20:15
09-06-22 21:13:28-INFO-pax    5  got to dest at                           2022-06-09 21:20:35
09-06-22 21:13:28-INFO-pax    9  got

In [27]:
sim.res[0].platform

,fare,name,batch_time,base_fare,min_fare,profit
1,1.2,NaN,NaN,2,2,18.879
